# Preparing the data
This notebook shows how to tile up RGB and crown data ready for training.

## Mount drive to access data and install *detectree2* package.

In [7]:
from google.colab import drive
drive.mount('/content/drive')
!pip install git+https://github.com/PatBall1/detectree2.git@jb/july24

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
  Cloning https://github.com/PatBall1/detectree2.git (to revision jb/july24) to /tmp/pip-req-build-lvjpyiwj
  Running command git clone --filter=blob:none --quiet https://github.com/PatBall1/detectree2.git /tmp/pip-req-build-lvjpyiwj
  Running command git checkout -b jb/july24 --track origin/jb/july24
  Switched to a new branch 'jb/july24'
  Branch 'jb/july24' set up to track remote branch 'jb/july24' from 'origin'.
  Resolved https://github.com/PatBall1/detectree2.git to commit c2267505f31ca3956d3c1ba17ffcd1faf23076f3
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-install-0s2eo4dc/detectron2_1cfc718b4490423596f88a429153d168
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-install-0s2eo4dc/detectron2_1cfc718b4490423596f88a429153

## Set parameters for tiling

In [16]:
from detectree2.preprocessing.tiling import tile_data, to_traintest_folders
import rasterio
import geopandas as gpd
import shutil
import time

# Set tiling parameters
buffer = 30
tile_width = 40
tile_height = 40
threshold = 0.6
appends = str(tile_width) + "_" + str(buffer) + "_" + str(threshold)

# dtype_bool requires True: BCI_2019, Paracou

## Tile up the data
Function to tile up the data into managable training chunks. This function has some issues around the encoding of the input raster. ```dtype_bool``` should be switched if black tiles are being produced. A recommended threshold is ~0.5 but it depends on volume of available data  (with abundant, dense crown data, a sticter threshold may be preferable).

In [ ]:

### PARACOU 2016
out_dir = site_path + '/tilesISPRS_' + appends + "/"

# Remove existing tile directory
#shutil.rmtree(out_dir, True)

# Read in the tiff file
data = rasterio.open(img_path)

# Read in crowns (then filter by an attribute?)
crowns = gpd.read_file(crown_path)
crowns = crowns.to_crs(data.crs.data)

tile_data_train(data, out_dir, buffer, tile_width, tile_height, crowns, threshold)
#to_traintest_folders(out_dir, out_dir, test_frac=0, folds=5)

/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


Tiling complete


In [ ]:
crowns[~crowns.is_valid]

,fid_1,Site,PlotOrg,PlotNum,SubPlot,LocalID,CensusYear,CodeAlive,Family,Genus_Species,...,Lianas,StartDate,EndDate,GroundValid,Creator,Comments,BaseLayer,IDStatus,DBHest,geometry
219,7471,Paracou,CIRAD,5.0,3.0,398.0,2015.0,True,Clusiaceae,Symphonia_globulifera,...,False,NaN,NaN,True,Greg Vincent,398,Lidar2016,NaN,NaN,"MULTIPOLYGON (((286188.347 583007.643, 286189...."
4376,3281,Paracou,External,NaN,NaN,NaN,NaN,NaN,NaN,NA_NA,...,NaN,NaN,NaN,NaN,Manon,NaN,NaN,NaN,NaN,"MULTIPOLYGON (((286537.890 583781.031, 286536...."


In [19]:
### PARACOU 2016
site_path = "/content/drive/MyDrive/WORK/detectree2/data/Paracou"
img_path = site_path + "/rgb/2016/Paracou_RGB_2016_10cm.tif"
crown_path = site_path + "/crowns/240717_full_detectree2016.gpkg"
#crown_path = "/content/drive/MyDrive/WORK/detectree2/data/Paracou_multitemp/crowns/crowns_train_2016.gpkg" # Excludes multitemp test crowns
out_dir = "/content/drive/MyDrive/WORK/detectree2/data/Paracou_TESTTESTTEST" + '/tiles_' + appends + "/"

# Remove existing tile directory
shutil.rmtree(out_dir, True)

# Read in the tiff file
data = rasterio.open(img_path)

# Read in crowns (then filter by an attribute?)
crowns = gpd.read_file(crown_path)
crowns = crowns[crowns.is_valid]
crowns = crowns.to_crs(data.crs.data)

start_time = time.time()
tile_data(img_path, out_dir, buffer, tile_width, tile_height, crowns, threshold, dtype_bool = True)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds")

# OLD TIME: 420.24s, 669.93s
# NEW TIME: 470.47s (threadpool), 335.10 (taskpool), 299.08 (taskpool)

to_traintest_folders(out_dir, out_dir, test_frac=0, folds=5)
#/content/drive/MyDrive/WORK/detectree2/data/Paracou/crowns/240717_full_detectree2016.gpkg

/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


Elapsed time: 227.692321062088 seconds


In [ ]:
### PARACOU 2019
site_path = "/content/drive/Shareddrives/detectree2/data/Paracou"
img_path = site_path + "/rgb/2019/Paracou_RGB_2019.tif"
#crown_path = site_path + "/crowns/220908_Paracou2016.gpkg"
crown_path = "/content/drive/Shareddrives/detectree2/data/Paracou_multitemp/crowns/crowns_train_2019.gpkg" # Excludes multitemp test crowns
out_dir = "/content/drive/Shareddrives/detectree2/data/Paracou_multitemp" + '/tiles2019_' + appends + "/"

# Remove existing tile directory
shutil.rmtree(out_dir, True)

# Read in the tiff file
data = rasterio.open(img_path)

# Read in crowns (then filter by an attribute?)
crowns = gpd.read_file(crown_path)
crowns = crowns.to_crs(data.crs.data)

tile_data_train(data, out_dir, buffer, tile_width, tile_height, crowns, threshold, dtype_bool = True)
to_traintest_folders(out_dir, out_dir, test_frac=0, folds=5)

/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [5]:
### DANUM
site_path = "/content/drive/MyDrive/WORK/detectree2/data/Danum"
img_path = site_path + "/rgb/Dan_2014_RGB_project_to_CHM.tif"
crown_path = site_path + "/crowns/Danum.gpkg"
#out_dir = site_path + '/tiles_' + appends + "/"
out_dir = "/content/drive/MyDrive/WORK/detectree2/data/Paracou_TESTTESTTEST1" + '/Dan_' + appends + "/"

# Remove existing tile directory
shutil.rmtree(out_dir, True)

# Read in the tiff file
data = rasterio.open(img_path)

# Read in crowns (then filter by an attribute?)
crowns = gpd.read_file(crown_path)
#crowns = crowns[crowns.conf==1]
crowns = crowns.to_crs(data.crs.data)

start_time = time.time()
tile_data(img_path, out_dir, buffer, tile_width, tile_height, crowns, threshold)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds")

# OLD TIME: 147.76s
# NEW TIME: 203.72s, 143.25 (taskpool), 93.34s (taskpool)

to_traintest_folders(out_dir, out_dir, test_frac=0, folds=5)

/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


Elapsed time: 93.33679580688477 seconds


In [ ]:
### SEPILOK EAST
site_path = "/content/drive/Shareddrives/detectree2/data/Sepilok"
img_path = site_path + "/rgb/RCD105_MA14_21_orthomosaic_20141023_reprojected_full_res.tif"
crown_path = site_path + "/crowns/SepilokEast.gpkg"
#out_dir = site_path + '/tilesE_' + appends + "/"
out_dir = "/content/drive/Shareddrives/detectree2/data/Paracou_multitemp" + '/SepE_' + appends + "/"

# Remove existing tile directory
shutil.rmtree(out_dir, True)

# Read in the tiff file
data = rasterio.open(img_path)

# Read in crowns (then filter by an attribute?)
crowns = gpd.read_file(crown_path)
#crowns = crowns[crowns.conf==1]
crowns = crowns.to_crs(data.crs.data)

tile_data_train(data, out_dir, buffer, tile_width, tile_height, crowns, threshold, dtype_bool = False)
to_traintest_folders(out_dir, out_dir, test_frac=0, folds=5)

/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [ ]:
### SEPILOK West
site_path = "/content/drive/Shareddrives/detectree2/data/Sepilok"
img_path = site_path + "/rgb/RCD105_MA14_21_orthomosaic_20141023_reprojected_full_res.tif"
crown_path = site_path + "/crowns/SepilokWest.gpkg"
#out_dir = site_path + '/tilesW_' + appends + "/"
out_dir = "/content/drive/Shareddrives/detectree2/data/Paracou_multitemp" + '/SepW_' + appends + "/"

# Remove existing tile directory
shutil.rmtree(out_dir, True)

# Read in the tiff file
data = rasterio.open(img_path)

# Read in crowns (then filter by an attribute?)
crowns = gpd.read_file(crown_path)
#crowns = crowns[crowns.conf==1]
crowns = crowns.to_crs(data.crs.data)

tile_data_train(data, out_dir, buffer, tile_width, tile_height, crowns, threshold, dtype_bool = False)
to_traintest_folders(out_dir, out_dir, test_frac=0, folds=5)

/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [ ]:
### BCI50ha
### BCI 50 ha
site_path = "/content/drive/Shareddrives/detectree2/data/BCI_50ha"
img_path = site_path + "/rgb/2015.06.10_07cm_ORTHO.tif"
crown_path = site_path + "/crowns/BCI_CrownData_2014-10-02_KCaligned/BCI_All_Crown_Data_10ha_50ha.shp"
out_dir = site_path + '/tiles_' + appends + "/"

# Remove existing tile directory
shutil.rmtree(out_dir, True)

# Read in the tiff file
data = rasterio.open(img_path)

# Read in crowns (then filter by an attribute?)
crowns = gpd.read_file(crown_path)
#crowns = crowns[crowns.conf==1]
crowns = crowns.to_crs(data.crs.data)

tile_data_train(data, out_dir, buffer, tile_width, tile_height, crowns, threshold, dtype_bool = True)
to_traintest_folders(out_dir, out_dir, test_frac=0, folds=5)

/usr/local/lib/python3.8/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [ ]:
import glob
import re
from pathlib import Path
from detectree2.preprocessing.tiling import to_traintest_folders

### PARACOU UAV NEW
site_path = "/content/drive/Shareddrives/detectree2/data/Paracou_multitemp"
img_folder = site_path + "/rgb/4D_rect/"
images = glob.glob(img_folder + "*.tif")
images.sort()
crown_path = site_path + "/crowns/crowns_train_2020.gpkg"
out_dir = site_path + "/tilesUAV_" + appends + "/"

#print(images)

## Remove existing tile directory
#shutil.rmtree(out_dir, True)

crowns = gpd.read_file(crown_path)

for image in images:
    data = rasterio.open(image)
    p = Path(image)
    date = re.search(r'\d{8}', image).group()
    print(date)
    foldername = "tilesUAV_" + date + "_" + appends
    #print(filename)
    out_path = Path(site_path) / foldername
    #print(out_path)
    tile_data_train(data, out_path, buffer, tile_width, tile_height, crowns, threshold, dtype_bool = True)
    to_traintest_folders(out_path, out_path, test_frac=0.0, folds=5)

20201023
20201105
20201123
20201214
20210105
20210118
20210208
20210303
20210316
20210406
20210428
20210511
20210630
20210706
20210726
20210817
20210907
20210928
20211018
20211110
20211123
20211213
20220106
20220124
20220223
20220317
20220406
20220426
20220517
20220608
20220628
20220719
20220809
20220830
20220920
20221011
20221108
20221129
20221214
20230103
20230117
20230216
20230228
20230314
20230328
20230420


## Send geojson to train/test folders
Send geojsons to train folder (with folds for k-fold cross validation) and test folder. Training tiles will automatically be remove if there is any overlap with a test tile.

In [ ]:
from detectree2.preprocessing.tiling import to_traintest_folders
#out_folder = out_dir
to_traintest_folders(out_dir, out_dir, test_frac=0.0, folds=5)

## Visualise training data

Need to edit to register properly. Fixed in training script

In [20]:
# Let's look at our training image and annos for our geojson
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.utils.visualizer import Visualizer
from detectree2.models.train import combine_dicts, register_train_data
import random
import cv2
from PIL import Image



#name = "GAN"
#train_location = "/content/drive/MyDrive/WORK/detectree2/data/" + name + "/tiles_" + appends + "/train"
name = "Paracou"
train_location = out_dir + "/train"
dataset_dicts = combine_dicts(train_location, 5)
trees_metadata = MetadataCatalog.get(name + "_train")
#dataset_dicts = get_tree_dicts("./")
for d in dataset_dicts:
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=trees_metadata, scale=0.8)
    out = visualizer.draw_dataset_dict(d)
    image = cv2.cvtColor(out.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB)
    display(Image.fromarray(image))

Output hidden; open in https://colab.research.google.com to view.